In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sun Sep 29 10:26:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


## Install Segment Anything Model (SAM) and other dependencies

In [4]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [5]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 40.8 MB/s eta 0:00:00


### Download SAM weights

In [6]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [7]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


## Load Model

In [8]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [9]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


### Utils Supporting Dataset Processing

A couple of helper functions that, unfortunately, we have to write ourselves to facilitate the processing of COCO annotations.

In [10]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Union, Optional
from dataclasses_json import dataclass_json
from supervision import Detections


@dataclass_json
@dataclass
class COCOCategory:
    id: int
    name: str
    supercategory: str


@dataclass_json
@dataclass
class COCOImage:
    id: int
    width: int
    height: int
    file_name: str
    license: int
    date_captured: str
    coco_url: Optional[str] = None
    flickr_url: Optional[str] = None


@dataclass_json
@dataclass
class COCOAnnotation:
    id: int
    image_id: int
    category_id: int
    segmentation: List[List[float]]
    area: float
    bbox: Tuple[float, float, float, float]
    iscrowd: int


@dataclass_json
@dataclass
class COCOLicense:
    id: int
    name: str
    url: str


@dataclass_json
@dataclass
class COCOJson:
    images: List[COCOImage]
    annotations: List[COCOAnnotation]
    categories: List[COCOCategory]
    licenses: List[COCOLicense]


def load_coco_json(json_file: str) -> COCOJson:
    import json

    with open(json_file, "r") as f:
        json_data = json.load(f)

    return COCOJson.from_dict(json_data)


class COCOJsonUtility:
    @staticmethod
    def get_annotations_by_image_id(coco_data: COCOJson, image_id: int) -> List[COCOAnnotation]:
        return [annotation for annotation in coco_data.annotations if annotation.image_id == image_id]

    @staticmethod
    def get_annotations_by_image_path(coco_data: COCOJson, image_path: str) -> Optional[List[COCOAnnotation]]:
        image = COCOJsonUtility.get_image_by_path(coco_data, image_path)
        if image:
            return COCOJsonUtility.get_annotations_by_image_id(coco_data, image.id)
        else:
            return None

    @staticmethod
    def get_image_by_path(coco_data: COCOJson, image_path: str) -> Optional[COCOImage]:
        for image in coco_data.images:
            if image.file_name == image_path:
                return image
        return None

    @staticmethod
    def annotations2detections(annotations: List[COCOAnnotation]) -> Detections:
        class_id, xyxy = [], []

        for annotation in annotations:
            x_min, y_min, width, height = annotation.bbox
            class_id.append(annotation.category_id)
            xyxy.append([
                x_min,
                y_min,
                x_min + width,
                y_min + height
            ])

        return Detections(
            xyxy=np.array(xyxy, dtype=int),
            class_id=np.array(class_id, dtype=int)
        )

### Download Dataset from Roboflow - run all images (train valid and test ) for both rgb and thermal converted min max version

In [14]:
mask_predictor = SamPredictor(sam)

In [15]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow
import supervision as sv
import random

def load_dataset(api_key, workspace, project_name, version_number):
    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace).project(project_name)
    version = project.version(version_number)
    return version.download("coco")

def get_image_list_and_annotations(dataset_location, subdirectory):
    annotations_file_path = os.path.join(dataset_location, subdirectory, "_annotations.coco.json")
    coco_data = load_coco_json(json_file=annotations_file_path)
    image_list = [image.file_name for image in coco_data.images]
    return image_list, coco_data

def process_images(image_list, dataset_location, subdirectory, coco_data, output_directory):
    df_segments = pd.DataFrame(columns=['Image Name', 'Area'])
    masked_directory = os.path.join(output_directory, subdirectory + "_masked")
    os.makedirs(masked_directory, exist_ok=True)

    for image_name in image_list:
        image_path = os.path.join(dataset_location, subdirectory, image_name)
        image = cv2.imread(image_path)
        if image is None:
            continue

        processed_image, area = process_single_image(image, coco_data, image_name)
        if processed_image is not None:
            save_path = os.path.join(masked_directory, f"{image_name}_mask.png")
            cv2.imwrite(save_path, processed_image)
            df_segments.loc[len(df_segments)] = [image_name, area]

    return df_segments

def process_single_image(image, coco_data, image_name):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=image_name)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id -= 1

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(box=ground_truth.xyxy[0], multimask_output=True)

    if len(masks) > 0:
        mask = masks[0]
        binary_mask = np.invert((mask > 0).astype(np.uint8) * 255)
        area = np.sum(binary_mask == 255)
        return binary_mask, area
    return None, 0

# Main execution

#RGB
# api_key = "aCFGMWNyipynQqvPzfaM"
# workspace = "citrus-a0lk2"
# project_name = "segment-performance-measure-2"
# version_number = 2

#THERMAL
api_key= "aCFGMWNyipynQqvPzfaM"
workspace = "citrus-a0lk2"
project_name = "performance_measure_thermal"
version_number = 1


dataset = load_dataset(api_key, workspace, project_name, version_number)
dataset_location = dataset.location
subdirectories = ["train", "valid", "test"]  # List of subdirectories
# output_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM"
output_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_MANUAL_THERMAL"

all_segments = pd.DataFrame()

for subdirectory in subdirectories:
    images, coco_data = get_image_list_and_annotations(dataset_location, subdirectory)
    df_segments = process_images(images, dataset_location, subdirectory, coco_data, output_directory)
    all_segments = pd.concat([all_segments, df_segments], ignore_index=True)

print(all_segments)


loading Roboflow workspace...
loading Roboflow project...
                                           Image Name    Area
0   FLIR1616_jpg.rf.23655aff9a1a3dea8d49ddeeda4ae9...  661068
1   FLIR1425_jpg.rf.0363c9af57273af40c8464bdcdbbbb...  713035
2   FLIR1958_jpg.rf.326eebd8b60020d28631966a3f60ae...  643093
3   FLIR1080_jpg.rf.3e06ec5b6b3458dc58f344db434618...  754822
4   FLIR1651_jpg.rf.20f8c6dd41dd0a33776ba64c31951b...  649038
..                                                ...     ...
94  FLIR1999_jpg.rf.ab9e89cf1c77cffa9921b5dbb4a4a9...  664859
95  FLIR1311_jpg.rf.074c87bc07f5a5a81fb1a87b439b3e...  703043
96  FLIR1914_jpg.rf.19fcbe0cce8f4c23ab0e17843f1759...  703159
97  FLIR1407_jpg.rf.a7a0ef9d66f7ef6a93cbc39ce99b0d...  682167
98  FLIR1851_jpg.rf.3f386375898e6498b0b24d897886d3...  690299

[99 rows x 2 columns]


In [ ]:
%cd {HOME}

import roboflow
from roboflow import Roboflow


/content


In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")




loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Segment-performance-measure-2-2 in coco:: 100%|██████████| 108/108 [00:00<00:00, 139.79it/s]


In [ ]:
import os

DATA_SET_SUBDIRECTORY = "valid"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

In [ ]:
coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

CLASSES = [
    category.name
    for category
    in coco_data.categories
    if category.supercategory != 'none'
]

IMAGES = [image.file_name for image in coco_data.images]


In [ ]:
CLASSES

['mandarin']

In [ ]:
# set random seed to allow easy reproduction of the experiment

import random
random.seed(10)

In [ ]:
import cv2
import supervision as sv

In [ ]:
EXAMPLE_IMAGE_NAME = random.choice(IMAGES)
EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)

# load dataset annotations
annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)

# small hack - coco numerate classes from 1, model from 0 + we drop first redundant class from coco json
ground_truth.class_id = ground_truth.class_id - 1

# load image
image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

# initiate annotator
bounding_box_annotator = sv.BoundingBoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)


# annotate ground truth
annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

# run SAM inference
mask_predictor.set_image(image_rgb)

masks, scores, logits = mask_predictor.predict(
    box=ground_truth.xyxy[0],
    multimask_output=True
)

detections = sv.Detections(
    xyxy=sv.mask_to_xyxy(masks=masks),
    mask=masks
)
detections = detections[detections.area == np.max(detections.area)]

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[annotated_frame_ground_truth, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)




In [ ]:
print("Total images:", len(IMAGES))


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow

In [ ]:
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Define paths for 'valid' subdirectory
DATA_SET_SUBDIRECTORY = "valid"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)


CLASSES = [
    category.name
    for category
    in coco_data.categories
    if category.supercategory != 'none'
]

IMAGES = [image.file_name for image in coco_data.images]

In [ ]:

# DataFrame for storing image names and areas
df_segments_valid = pd.DataFrame(columns=['Image Name', 'Area'])

import random
random.seed(10)

import supervision as sv

# Process each image
for EXAMPLE_IMAGE_NAME in IMAGES:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id = ground_truth.class_id - 1

    # Annotate using the new BoxAnnotator
    bounding_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],
        multimask_output=True
    )

    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )
    max_area = np.max([np.sum(mask) for mask in masks])
    detections = detections[detections.area == max_area]

    annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
    save_path = f"/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/{EXAMPLE_IMAGE_NAME}_annotated.png"
    cv2.imwrite(save_path, annotated_image)

    # Update the DataFrame with new data
    df_segments_valid.loc[len(df_segments_valid)] = [EXAMPLE_IMAGE_NAME, max_area]

# Display the DataFrame
print(df_segments_valid)


run again save the binary mask valid

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow
import supervision as sv
import random

# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")

# Define paths for 'valid' subdirectory
DATA_SET_SUBDIRECTORY = "valid"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

# DataFrame for storing image names and areas
df_segments_valid = pd.DataFrame(columns=['Image Name', 'Area'])

random.seed(10)


# Path for the new 'masked' folder
masked_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/masked"
os.makedirs(masked_directory, exist_ok=True)  # Ensure the directory exists


# Process each image in the 'valid' directory
for EXAMPLE_IMAGE_NAME in [image.file_name for image in coco_data.images]:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    if image_bgr is None:
        continue  # Skip if the image cannot be loaded

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id -= 1

    # Annotate using the BoxAnnotator
    bounding_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],
        multimask_output=True
    )

    # Save only the first mask if masks are available
    if len(masks) > 0:
        first_mask = masks[0]  # Take the first mask
        binary_mask = np.invert((first_mask > 0).astype(np.uint8) * 255)
        binary_mask_save_path = os.path.join(masked_directory, f"{EXAMPLE_IMAGE_NAME}_mask.png")
        cv2.imwrite(binary_mask_save_path, binary_mask)

    max_area = np.max([np.sum(first_mask) for first_mask in [masks[0]]] if len(masks) > 0 else [0])
    df_segments_train.loc[len(df_segments_train)] = [EXAMPLE_IMAGE_NAME, max_area]

# Display the DataFrame
print(df_segments_train)


loading Roboflow workspace...
loading Roboflow project...
                                           Image Name     Area
0   1D3A5563_JPG.rf.18495fb53856b511f81300816c2ce8...  6446404
1   1D3A3200_JPG.rf.29aeb7444ee567536ac2bc7b56d457...  1774001
2   1D3A3460_JPG.rf.2e7d2b8cf9f4bdf35a3cdd6ce03858...  8937229
3   1D3A5385_JPG.rf.2fe3c57d1c45685b6ad32e1464a6b3...  6082396
4   1D3A5496_JPG.rf.0ca7bf0b4bee5c54f37d601c10ea46...  5152810
..                                                ...      ...
95  1D3A4273_JPG.rf.9fa46b0866d5b88ffa0cadfcd63123...  6208076
96  1D3A5458_JPG.rf.93b91aedf33530af08a4f59b4d2c69...  8407367
97  1D3A5619_JPG.rf.f549bbbce325984fa7d7412fa636af...  4880941
98  1D3A5091_JPG.rf.d108d32f9115ecb2529bec341a12d2...  6191915
99  1D3A4408_JPG.rf.98dc9188aa1f6681359695533a42a7...  4812010

[100 rows x 2 columns]


In [ ]:
# Define paths for 'test' subdirectory
DATA_SET_SUBDIRECTORY = "test"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)


CLASSES = [
    category.name
    for category
    in coco_data.categories
    if category.supercategory != 'none'
]

IMAGES = [image.file_name for image in coco_data.images]

In [ ]:
# Define the directory for saving binary masks
mask_save_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/maskedtest"
os.makedirs(mask_save_directory, exist_ok=True)  # Ensure the directory exists

# Assuming 'masks' is a list of NumPy arrays where each element is a binary mask
# Let's say 'masks' is obtained from your mask predictor
for idx, EXAMPLE_IMAGE_NAME in enumerate(IMAGES):
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Assuming mask_predictor is set up as shown in your previous messages
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],  # Ensure this is correctly set up
        multimask_output=True
    )

    # Process each mask
    for mask_idx, mask in enumerate(masks):
        # Convert mask to binary format
        binary_mask = (mask > 0).astype(np.uint8) * 255  # Ensure mask is binary {0, 255}

        # Save the binary mask to file
        mask_filename = f"{EXAMPLE_IMAGE_NAME}_mask_{mask_idx}.png"
        mask_path = os.path.join(mask_save_directory, mask_filename)
        cv2.imwrite(mask_path, binary_mask)

print("All binary masks have been generated and saved.")


In [ ]:


# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")

# Define paths for 'test' subdirectory
DATA_SET_SUBDIRECTORY = "test"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

# DataFrame for storing image names and areas
df_segments_test = pd.DataFrame(columns=['Image Name', 'Area'])

random.seed(10)

# Process each image in the 'test' directory
for EXAMPLE_IMAGE_NAME in [image.file_name for image in coco_data.images]:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    if image_bgr is None:
        continue  # Skip if the image cannot be loaded

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id = ground_truth.class_id - 1

    # Annotate using the BoxAnnotator
    bounding_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],
        multimask_output=True
    )

    max_area = np.max([np.sum(mask) for mask in masks])
    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )
    detections = detections[detections.area == max_area]

    annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
    save_path = f"/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/{DATA_SET_SUBDIRECTORY}_{EXAMPLE_IMAGE_NAME}_annotated.png"
    cv2.imwrite(save_path, annotated_image)

    # Update the DataFrame with new data
    df_segments_test.loc[len(df_segments_test)] = [EXAMPLE_IMAGE_NAME, max_area]

# Display the DataFrame
print(df_segments_test)


run again but save the binary mask - test

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow
import supervision as sv
import random

# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")

# Define paths for 'test' subdirectory
DATA_SET_SUBDIRECTORY = "test"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

# DataFrame for storing image names and areas
df_segments_test = pd.DataFrame(columns=['Image Name', 'Area'])

random.seed(10)


# Path for the new 'masked' folder
masked_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/masked"
os.makedirs(masked_directory, exist_ok=True)  # Ensure the directory exists


# Process each image in the 'test' directory
for EXAMPLE_IMAGE_NAME in [image.file_name for image in coco_data.images]:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    if image_bgr is None:
        continue  # Skip if the image cannot be loaded

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id -= 1

    # Annotate using the BoxAnnotator
    bounding_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],
        multimask_output=True
    )

    # Save only the first mask if masks are available
    if len(masks) > 0:
        first_mask = masks[0]  # Take the first mask
        binary_mask = np.invert((first_mask > 0).astype(np.uint8) * 255)
        binary_mask_save_path = os.path.join(masked_directory, f"{EXAMPLE_IMAGE_NAME}_mask.png")
        cv2.imwrite(binary_mask_save_path, binary_mask)

    max_area = np.max([np.sum(first_mask) for first_mask in [masks[0]]] if len(masks) > 0 else [0])
    df_segments_train.loc[len(df_segments_train)] = [EXAMPLE_IMAGE_NAME, max_area]

# Display the DataFrame
print(df_segments_train)


loading Roboflow workspace...
loading Roboflow project...
                                           Image Name      Area
0   1D3A5563_JPG.rf.18495fb53856b511f81300816c2ce8...   6446404
1   1D3A3200_JPG.rf.29aeb7444ee567536ac2bc7b56d457...   1774001
2   1D3A3460_JPG.rf.2e7d2b8cf9f4bdf35a3cdd6ce03858...   8937229
3   1D3A5385_JPG.rf.2fe3c57d1c45685b6ad32e1464a6b3...   6082396
4   1D3A5496_JPG.rf.0ca7bf0b4bee5c54f37d601c10ea46...   5152810
..                                                ...       ...
74  1D3A4538_JPG.rf.8d2a5b50bd06d18a974472115344d4...   6567573
75  1D3A4875_JPG.rf.dc3823d411c57cc86fdd9c1453f8b3...  10271194
76  1D3A4312_JPG.rf.f88435080170bc2a89f8f807e184cb...   4492194
77  1D3A4884_JPG.rf.1a4655fd29878dfe6a90508e7b5f2c...   4881794
78  1D3A3277_JPG.rf.d3705c2db4d238fe28f509670ab1da...   6061895

[79 rows x 2 columns]


In [ ]:
# Define paths for 'train' subdirectory
DATA_SET_SUBDIRECTORY = "train"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)


CLASSES = [
    category.name
    for category
    in coco_data.categories
    if category.supercategory != 'none'
]

IMAGES = [image.file_name for image in coco_data.images]

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow
import supervision as sv
import random

# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")

# Define paths for 'train' subdirectory
DATA_SET_SUBDIRECTORY = "train"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

# DataFrame for storing image names and areas
df_segments_train = pd.DataFrame(columns=['Image Name', 'Area'])

random.seed(10)

# Process each image in the 'train' directory
for EXAMPLE_IMAGE_NAME in [image.file_name for image in coco_data.images]:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    if image_bgr is None:
        continue  # Skip if the image cannot be loaded

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id = ground_truth.class_id - 1

    # Annotate using the BoxAnnotator
    bounding_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],
        multimask_output=True
    )

    max_area = np.max([np.sum(mask) for mask in masks])
    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )
    detections = detections[detections.area == max_area]

    annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
    save_path = f"/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/{DATA_SET_SUBDIRECTORY}_{EXAMPLE_IMAGE_NAME}_annotated.png"
    cv2.imwrite(save_path, annotated_image)

    # Update the DataFrame with new data
    df_segments_train.loc[len(df_segments_train)] = [EXAMPLE_IMAGE_NAME, max_area]

# Display the DataFrame
print(df_segments_train)


run again but save the binary mask - train

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow
import supervision as sv
import random

# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="aCFGMWNyipynQqvPzfaM")
project = rf.workspace("citrus-a0lk2").project("segment-performance-measure-2")
version = project.version(2)
dataset = version.download("coco")

# Define paths for 'train' subdirectory
DATA_SET_SUBDIRECTORY = "train"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

# DataFrame for storing image names and areas
df_segments_train = pd.DataFrame(columns=['Image Name', 'Area'])

random.seed(10)


# Path for the new 'masked' folder
masked_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/masked"
os.makedirs(masked_directory, exist_ok=True)  # Ensure the directory exists


# Process each image in the 'train' directory
for EXAMPLE_IMAGE_NAME in [image.file_name for image in coco_data.images]:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, EXAMPLE_IMAGE_NAME)
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    if image_bgr is None:
        continue  # Skip if the image cannot be loaded

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=EXAMPLE_IMAGE_NAME)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id -= 1

    # Annotate using the BoxAnnotator
    bounding_box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
    annotated_frame_ground_truth = bounding_box_annotator.annotate(scene=image_bgr.copy(), detections=ground_truth)

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(
        box=ground_truth.xyxy[0],
        multimask_output=True
    )

    # Save only the first mask if masks are available
    if len(masks) > 0:
        first_mask = masks[0]  # Take the first mask
        binary_mask = np.invert((first_mask > 0).astype(np.uint8) * 255)
        binary_mask_save_path = os.path.join(masked_directory, f"{EXAMPLE_IMAGE_NAME}_mask.png")
        cv2.imwrite(binary_mask_save_path, binary_mask)

    max_area = np.max([np.sum(first_mask) for first_mask in [masks[0]]] if len(masks) > 0 else [0])
    df_segments_train.loc[len(df_segments_train)] = [EXAMPLE_IMAGE_NAME, max_area]

# Display the DataFrame
print(df_segments_train)


                                           Image Name     Area
0   1D3A5563_JPG.rf.18495fb53856b511f81300816c2ce8...  6446404
1   1D3A3200_JPG.rf.29aeb7444ee567536ac2bc7b56d457...  1774001
2   1D3A3460_JPG.rf.2e7d2b8cf9f4bdf35a3cdd6ce03858...  8937229
3   1D3A5385_JPG.rf.2fe3c57d1c45685b6ad32e1464a6b3...  6082396
4   1D3A5496_JPG.rf.0ca7bf0b4bee5c54f37d601c10ea46...  5152810
..                                                ...      ...
65  1D3A5378_JPG.rf.9dbdcc6b914762c8a038d4c3b3d0a3...  9830132
66  1D3A4555_JPG.rf.f7dcbca67442d97a55488e4b6c620c...  5791913
67  1D3A5388_JPG.rf.f79ad7acf3361070aa72d0ef778702...  4540062
68  1D3A4481_JPG.rf.fccd6779d03ee02a4fb07e7cc0415a...  8888796
69  1D3A4323_JPG.rf.e6029d9f5d2d9be719cb9bf57e9c01...  4636828

[70 rows x 2 columns]


In [ ]:
import pandas as pd

# Assuming df_segments_valid, df_segments_test, df_segments_train are already defined
# Combine all three DataFrames into one
df_segments_combined = pd.concat([df_segments_train, df_segments_valid, df_segments_test], ignore_index=True)

# Display the combined DataFrame
print(df_segments_combined)


In [ ]:
import os
import pandas as pd

# Assuming df_segments is your DataFrame
save_path = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/mandarins_data.xlsx"

# Create the directory if it does not exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the DataFrame to an Excel file
df_segments_combined.to_excel(save_path, index=False)

print(f"DataFrame saved to {save_path}")


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Load the Excel file
df = pd.read_excel("/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/mandarins_data.xlsx")

# Add or update the 'Unet area' column
df['Unet area'] = 0

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract the first 8 characters from the image name to form the path to the UNet predicted image
    image_prefix = row['Image Name'][:8]
    unet_image_path = f"/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/unet/prediction/{image_prefix}.JPG"

    # Load the image in grayscale
    image = cv2.imread(unet_image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded correctly
    if image is not None:
        # Count pixels that are not white (value not equal to 255)
        non_white_pixels = np.count_nonzero(image != 255)
        df.at[index, 'Unet area'] = non_white_pixels
    else:
        print(f"Failed to load image at {unet_image_path}")

# Save the updated DataFrame back to Excel
df.to_excel("/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_2/mandarins_data.xlsx", index=False)

# Print the DataFrame to verify the output
print(df.head())


try to make the code more effecient

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from roboflow import Roboflow
import supervision as sv
import random

def load_dataset(api_key, workspace, project_name, version_number):
    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace).project(project_name)
    version = project.version(version_number)
    return version.download("coco")

def get_image_list_and_annotations(dataset_location, subdirectory):
    annotations_file_path = os.path.join(dataset_location, subdirectory, "_annotations.coco.json")
    coco_data = load_coco_json(json_file=annotations_file_path)
    image_list = [image.file_name for image in coco_data.images]
    return image_list, coco_data

def process_images(image_list, dataset_location, subdirectory, coco_data, output_directory):
    df_segments = pd.DataFrame(columns=['Image Name', 'Area'])
    masked_directory = os.path.join(output_directory, subdirectory + "_masked")
    os.makedirs(masked_directory, exist_ok=True)

    for image_name in image_list:
        image_path = os.path.join(dataset_location, subdirectory, image_name)
        image = cv2.imread(image_path)
        if image is None:
            continue

        processed_image, area = process_single_image(image, coco_data, image_name)
        if processed_image is not None:
            save_path = os.path.join(masked_directory, f"{image_name}_mask.png")
            cv2.imwrite(save_path, processed_image)
            df_segments.loc[len(df_segments)] = [image_name, area]

    return df_segments

def process_single_image(image, coco_data, image_name):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=image_name)
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)
    ground_truth.class_id -= 1

    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(box=ground_truth.xyxy[0], multimask_output=True)

    if len(masks) > 0:
        mask = masks[0]
        binary_mask = np.invert((mask > 0).astype(np.uint8) * 255)
        area = np.sum(binary_mask == 255)
        return binary_mask, area
    return None, 0

# Main execution
api_key = "aCFGMWNyipynQqvPzfaM"
workspace = "citrus-a0lk2"
project_name = "segment-performance-measure-2"
version_number = 2
dataset = load_dataset(api_key, workspace, project_name, version_number)
dataset_location = dataset.location
subdirectories = ["train", "valid", "test"]  # List of subdirectories
output_directory = "/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM"

all_segments = pd.DataFrame()

for subdirectory in subdirectories:
    images, coco_data = get_image_list_and_annotations(dataset_location, subdirectory)
    df_segments = process_images(images, dataset_location, subdirectory, coco_data, output_directory)
    all_segments = pd.concat([all_segments, df_segments], ignore_index=True)

print(all_segments)


loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 